In [ ]:
from smart_open import open
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize

# Load data for attackers

In [ ]:
data = pd.read_csv("../data/training_pruned_file_20162021.csv")

In [ ]:
players = data.loc[lambda x: x["position"] == "Attack"]

In [ ]:
sorted(players.columns.values)

## Select variables

In [ ]:
players_selected_vars = players[[
    "name",
    "player_id_api",
    "date_of_birth",
    "nationality",
    "team_name",
    'league_name',
     'league.season',
    # "error_lead_to_goal.total",
    # "saves_inside_box.total",
    "games.appearences",
    "games.lineups",
    "games.minutes",
    # "goals.conceded",
    # "goals.saves",
    # "penalty.saved",
    # "highest_market_value_in_eur",
    "market_value_in_eur_x",
    # "own_goals.total",
    "accurate_crosses.total",
    "big_chances_created.total",
    "big_chances_missed.total",
    "blocked_shots.total",
    "dribbled_past.total",
    "goals.goals",
    "offsides.total",
    "shots_off_target.total",
    "through_balls.total",
    "total_crosses.total",
    "shots.total",
    "shots.on",
    "goals.total_y",
    "goals.assists",
    "passes.key",
    "dribbles.attempts",
    "dribbles.success",
    "dribbles.past",
]].copy().rename({
    "team_name": "team",
    "league_name": "league",
    "league.season": "season",
    "games.appearences": "matches_played",
    "games.lineups": "matches_started",
    "games.minutes": "minutes",
    # "goals.conceded": "goals_against",
    # "goals.saves": "saves",
    # "saves_inside_box.total": "saves_inside_box",
    # "penalty.saved": "penalty_kicks_saves",
    # "own_goals.total": "own_goals_against",
    # "error_lead_to_goal.total": "errors_to_goal",
    "market_value_in_eur_x": "market_value",
}, axis=1)
players_selected_vars["90s_played"] = players_selected_vars["minutes"] / 90
# players_selected_vars["goals_against_90s"] = players_selected_vars["goals_against"] / players_selected_vars["90s_played"]
# players_selected_vars["shots_on_target_against"] = players_selected_vars["goals_against"] + players_selected_vars["saves"]
# players_selected_vars["save%"] = players_selected_vars["saves"] / players_selected_vars["shots_on_target_against"]
# players_selected_vars["age"] = pd.to_datetime(players_selected_vars["date_of_birth"]).map(lambda x: int((pd.Timestamp.today() - x).days / 365))
players_selected_vars["age"] = (26 - (players["league.season"] - players["date_of_birth"].str.slice(0, 4).astype(int)))
players_selected_vars["age2"] = players_selected_vars["age"] ** 2
# players_selected_vars["own_goals_against"] = players_selected_vars["own_goals_against"].fillna(0)
for per_90 in [
    # "shots_on_target_against",
    # "saves_inside_box",
    # "saves",
    # "penalty_kicks_saves",
    # "own_goals_against",
    # "goals_against",
    # "accurate_crosses.total",
    # "big_chances_created.total",
    # "big_chances_missed.total",
    # "blocked_shots.total",
    # "dribbled_past.total",
    # "goals.goals",
    # "offsides.total",
    # "shots_off_target.total",
    # "through_balls.total",
    # "total_crosses.total",
    # "shots.total",
    # "shots.on",
    # "goals.total_y",
    # "goals.assists",
    # "passes.key",
    # "dribbles.attempts",
    # "dribbles.success",
    # "dribbles.past",
]:
    players_selected_vars[per_90] = players_selected_vars[per_90] / players_selected_vars["90s_played"]


players_selected_vars.replace([np.inf, -np.inf], np.nan, inplace=True)
    
players_selected_vars = players_selected_vars.drop([
    "date_of_birth",
    "minutes",
], axis=1)

order = [
    "name",
    "player_id_api",
    "age",
    "age2",
    "nationality",
    "league",
    "team",
    "season",
    "matches_played",
    "matches_started",
    # "minutes",
    "90s_played",
    # "goals_against",
    # "own_goals_against",
    # "shots_on_target_against",
    # "saves",
    # "save%",
    # "saves_inside_box",
    # "penalty_kicks_saves",
    "market_value",
    "accurate_crosses.total",
    "big_chances_created.total",
    "big_chances_missed.total",
    "blocked_shots.total",
    "dribbled_past.total",
    "goals.goals",
    "offsides.total",
    "shots_off_target.total",
    "through_balls.total",
    "total_crosses.total",
    "shots.total",
    "shots.on",
    "goals.total_y",
    "goals.assists",
    "passes.key",
    "dribbles.attempts",
    "dribbles.success",
    "dribbles.past",
]
players_selected_vars = players_selected_vars[order + list(set(players_selected_vars.columns).difference(order))]
players_selected_vars

In [ ]:
players_selected_vars.groupby("age")["market_value"].mean().plot()

In [ ]:
from helpers import TEAM_TIERS

In [ ]:
players_selected_vars = pd.merge(players_selected_vars, TEAM_TIERS, left_on="team", right_index=True)

In [ ]:
mvs = players_selected_vars["market_value"].values
mvs = np.expand_dims(mvs, axis=1)
mvs = normalize(mvs, norm="max", axis=0)
mvs = np.squeeze(mvs)
players_selected_vars["normalised_market_value"] = mvs

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.dummy import DummyClassifier, DummyRegressor
from sklearn.impute import SimpleImputer
from sklearn.inspection import permutation_importance
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

In [ ]:
market_values = players_selected_vars[["player_id_api", "season", "market_value"]]
market_values["season"] = market_values["season"] +1
market_values

In [ ]:
players_selected_vars = pd.merge(players_selected_vars, market_values, on=["player_id_api", "season"], suffixes = ["", "_prev"])

## Transformation pipeline

In [ ]:
numeric_features = [
    "market_value_prev",
    "age",
    "age2",
    "matches_played",
    "matches_started",
    "90s_played",
    # "goals_against",
    # "own_goals_against",
    # "shots_on_target_against",
    # "saves",
    # "save%",
    # "saves_inside_box",
    # "penalty_kicks_saves",
    "accurate_crosses.total",
    "big_chances_created.total",
    "big_chances_missed.total",
    "blocked_shots.total",
    "dribbled_past.total",
    "goals.goals",
    "offsides.total",
    "shots_off_target.total",
    "through_balls.total",
    "total_crosses.total",
    "shots.total",
    "shots.on",
    "goals.total_y",
    "goals.assists",
    "passes.key",
    "dribbles.attempts",
    "dribbles.success",
    "dribbles.past",
]
numeric_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ]
)

categorical_features = [
    "nationality",
    "league",
    "team",
    "tier",
]
categorical_transformer = Pipeline(
    steps=[
        ("encoder", OneHotEncoder(handle_unknown="ignore"))
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

# Predict price bucket

In [ ]:
bins = [
    0.0025,
    0.005,
    0.01,
    0.025,
    0.05,
    0.1,
    0.25,
    0.5,
    1.0
]
y = np.digitize(players_selected_vars["normalised_market_value"], bins)
X = players_selected_vars

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
np.unique(y_train, return_counts=True)

In [ ]:
random = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("classifier", DummyClassifier(strategy="stratified")),
    ]
)
random.fit(X_train, y_train)

y_random = random.predict(X_test)

balanced_accuracy_score(y_test, y_random)

In [ ]:
np.unique(y_random, return_counts=True)

In [ ]:
clf = Pipeline(
    steps=[
        ("preprocessor", preprocessor), 
        ("classifier", LogisticRegression()),
    ]
)
clf.fit(X_train, y_train)

y_lr = clf.predict(X_test)

balanced_accuracy_score(y_test, y_lr)

In [ ]:
np.unique(y_test - y_lr, return_counts=True)

In [ ]:
np.unique(y_lr, return_counts=True)

In [ ]:
from sklearn.neural_network import MLPClassifier

mlp = Pipeline(
    steps=[
        ("preprocessor", preprocessor), 
        ("classifier", MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(16, 8, 4))),
    ]
)
mlp.fit(X_train, y_train)

y_mlp = mlp.predict(X_test)

balanced_accuracy_score(y_test, y_mlp)

## Features importance

In [ ]:
result = permutation_importance(
    clf, X_test, y_test, n_repeats=10, random_state=42, n_jobs=2
)

forest_importances = pd.Series(result.importances_mean, index=X_test.columns)

fig, ax = plt.subplots()
forest_importances.plot.bar(yerr=result.importances_std, ax=ax)
ax.set_title("Feature importances using permutation on full model")
ax.set_ylabel("Mean accuracy decrease")
fig.tight_layout()
plt.show()

# Predict market value

In [ ]:
def r2_adj(model, x,y):
    r2 = model.score(x,y)
    n = x.shape[0]
    p = x.shape[1]
    return 1-(1-r2)*(n-1)/(n-p-1)

In [ ]:
players_selected_vars.query("player_id_api == 18788")

In [ ]:
appearances = 15
X = players_selected_vars.loc[lambda x: np.logical_and(x["matches_played"] >= appearances, x["market_value"] != x["market_value_prev"])]
# y = players_selected_vars["normalised_market_value"]
y = np.log(players_selected_vars.loc[lambda x: np.logical_and(x["matches_played"] >= appearances, x["market_value"] != x["market_value_prev"])]["market_value"])
# y = players_selected_vars["market_value"]

In [ ]:
X_train = X.loc[X["season"] <= 2020]
X_test = X.loc[X["season"] == 2021]
y_train = y.loc[X["season"] <= 2020]
y_test = y.loc[X["season"] == 2021]
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
random_regression = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("regression", DummyRegressor()),
    ]
)
random_regression.fit(X_train, y_train)

random_regression.score(X_test, y_test), r2_adj(random_regression, X_test, y_test)

In [ ]:
regression = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("regression", LinearRegression()),
    ]
)
regression.fit(X_train, y_train)

regression.score(X_train, y_train), r2_adj(regression, X_train, y_train), regression.score(X_test, y_test), r2_adj(regression, X_test, y_test)

In [ ]:
from sklearn.neural_network import MLPRegressor

mlp_reg = Pipeline(
    steps=[
        ("preprocessor", preprocessor), 
        ("regression", MLPRegressor(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(8, 2))),
    ]
)
mlp_reg.fit(X_train, y_train)

mlp_reg.score(X_train, y_train), r2_adj(mlp_reg, X_train, y_train), mlp_reg.score(X_test, y_test), r2_adj(mlp_reg, X_test, y_test)

In [ ]:
from xgboost import XGBRegressor

xgboost_reg = Pipeline(
    steps=[
        ("preprocessor", preprocessor), 
        ("regression", XGBRegressor(n_estimators=500, max_depth=3, reg_lambda=50)),
    ]
)
xgboost_reg.fit(X_train, y_train)

xgboost_reg.score(X_train, y_train), r2_adj(xgboost_reg, X_train, y_train), xgboost_reg.score(X_test, y_test), r2_adj(xgboost_reg, X_test, y_test)

In [ ]:
###### from xgboost import XGBRegressor

xgboost_reg = Pipeline(
    steps=[
        ("preprocessor", preprocessor), 
        ("regression", XGBRegressor(n_estimators=500, max_depth=3, reg_lambda=50)),
    ]
)
xgboost_reg.fit(X_train, y_train)

print(xgboost_reg.score(X_train, y_train), r2_adj(xgboost_reg, X_train, y_train), xgboost_reg.score(X_test, y_test), r2_adj(xgboost_reg, X_test, y_test))

In [ ]:
reg_results = pd.DataFrame({
    "y": np.exp(y_test),
    "y_pred": np.exp(xgboost_reg.predict(X_test))
    # "y_pred": np.exp(regression.predict(X_test))
    # "y": y_test,
    # "y_pred": xgboost_reg.predict(X_test)
})
reg_results["diff"] = reg_results["y_pred"] - reg_results["y"]
reg_results["rel_error"] = reg_results["diff"] / reg_results["y"] * 100
reg_results["abs_rel_error"] = np.abs(reg_results["rel_error"])
reg_results = reg_results.sort_values("abs_rel_error")
reg_results

In [ ]:
plt.plot(
    reg_results["abs_rel_error"].rank() / len(reg_results["abs_rel_error"]) * 100,
    reg_results["abs_rel_error"].cumsum() / reg_results["abs_rel_error"].rank()
)
plt.xticks(np.linspace(0, 1, 11)*100)
plt.xlabel(f"% of players (attackers with at least {appearances} appearance(s))")
plt.ylabel("Error [%]")
plt.grid()
plt.show()

In [ ]:
np.floor(reg_results["abs_rel_error"])

In [ ]:
_x, _y = np.unique(np.floor(reg_results["abs_rel_error"]), return_counts=True)

In [ ]:
plt.plot(
    _x,
    _y.cumsum() / len(reg_results) * 100
)
plt.grid()
plt.xlabel("Error [%]")
plt.ylabel(f"% of players")
plt.title(f"Attackers with at least {appearances} appearance(s)")
plt.yticks(np.arange(0, 101, 10))
plt.xticks(np.arange(0, 201, 20))
plt.show()


In [ ]:
_ = plt.hist(reg_results["abs_rel_error"], bins=list(np.linspace(0, 101, 100)))

In [ ]:
# pd.merge(X_test.join(reg_results), X_train[["player_id_api", "age", "market_value", "season"]], on="player_id_api")

In [ ]:
# _ = plt.hist(reg_results["abs_rel_error"], bins=100)

In [ ]:
np.mean(reg_results["abs_rel_error"]), np.median(reg_results["abs_rel_error"])

In [ ]:
plt.scatter(y_train, regression.predict(X_train))
plt.scatter(y_train, random_regression.predict(X_train))
plt.scatter(y_train, xgboost_reg.predict(X_train))
# plt.scatter(y_train, mlp_reg.predict(X_train))
plt.axline((0, 0), (1, 1), c="k")
plt.xlim(np.floor(y_train.min())-0.1, np.ceil(y_train.max())+0.1)
plt.ylim(np.floor(y_train.min())-0.1, np.ceil(y_train.max())+0.1)
plt.xlabel("Observed")
plt.ylabel("Predicted")
_ = plt.show()

In [ ]:
plt.scatter(y_test, regression.predict(X_test))
plt.scatter(y_test, random_regression.predict(X_test))
plt.scatter(y_test, xgboost_reg.predict(X_test))
plt.axline((0, 0), (1, 1), c="k")
plt.xlim(np.floor(y_test.min())-0.1, np.ceil(y_test.max())+0.1)
plt.ylim(np.floor(y_test.min())-0.1, np.ceil(y_test.max())+0.1)
plt.xlabel("Observed")
plt.ylabel("Predicted")
_ = plt.show()

## Features importance

In [ ]:
result = permutation_importance(
    xgboost_reg, X_test, y_test, n_repeats=10, random_state=42, n_jobs=2
)

forest_importances = pd.Series(result.importances_mean, index=X_test.columns)

fig, ax = plt.subplots()
forest_importances.plot.bar(yerr=result.importances_std, ax=ax)
ax.set_title("Feature importances using permutation on full model")
ax.set_ylabel("Mean accuracy decrease")
fig.tight_layout()
plt.show()

# Predict change in market value between seasons

In [ ]:
before_after_seasons = pd.merge(
    players_selected_vars, 
    players_selected_vars, 
    on="player_id_api", 
    suffixes=["_before", "_after"]
).query("season_after - season_before == 1")

In [ ]:
numeric_features_ba = [
    f"{c}_{suffix}" for c in [
        "age",
        "matches_played",
        "matches_started",
        "90s_played",
        # "goals_against",
        # "own_goals_against",
        # "shots_on_target_against",
        # "saves",
        # "save%",
        # "saves_inside_box",
        # "penalty_kicks_saves",
        "accurate_crosses.total",
        "big_chances_created.total",
        "big_chances_missed.total",
        "blocked_shots.total",
        "dribbled_past.total",
        "goals.goals",
        "offsides.total",
        "shots_off_target.total",
        "through_balls.total",
        "total_crosses.total",
        "shots.total",
        "shots.on",
        "goals.total_y",
        "goals.assists",
        "passes.key",
        "dribbles.attempts",
        "dribbles.success",
        "dribbles.past",
    ]
    for suffix in ["before","after"]
]
numeric_transformer_ba = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ]
)

categorical_features_ba = [
    f"{c}_{suffix}" for c in [
        "nationality",
        "league",
        "team",
        "tier",
    ]
    for suffix in ["before","after"]
]
categorical_transformer_ba = Pipeline(
    steps=[
        ("encoder", OneHotEncoder(handle_unknown="ignore"))
    ]
)

preprocessor_ba = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer_ba, numeric_features_ba),
        ("cat", categorical_transformer_ba, categorical_features_ba),
    ]
)

In [ ]:
X = before_after_seasons
y_before = np.digitize(before_after_seasons["normalised_market_value_before"], bins)
y_after = np.digitize(before_after_seasons["normalised_market_value_after"], bins)
y = np.minimum(np.maximum(y_after - y_before, -1), 1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
random = Pipeline(
    steps=[
        ("preprocessor", preprocessor_ba),
        ("classifier", DummyClassifier(strategy="stratified")),
    ]
)
random.fit(X_train, y_train)

balanced_accuracy_score(y_test, random.predict(X_test))

In [ ]:
clf = Pipeline(
    steps=[
        ("preprocessor", preprocessor_ba), 
        ("classifier", LogisticRegression()),
    ]
)
clf.fit(X_train, y_train)

print(confusion_matrix(y_test, clf.predict(X_test), labels=[1, 0, -1]))

balanced_accuracy_score(y_test, clf.predict(X_test))

In [ ]:
from sklearn.neural_network import MLPClassifier

mlp = Pipeline(
    steps=[
        ("preprocessor", preprocessor_ba), 
        ("classifier", MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(32, 16))),
    ]
)
mlp.fit(X_train, y_train)

print(confusion_matrix(y_test, mlp.predict(X_test), labels=[1, 0, -1]))

balanced_accuracy_score(y_test, mlp.predict(X_test))

In [ ]:
from xgboost import XGBClassifier

xgboost = Pipeline(
    steps=[
        ("preprocessor", preprocessor_ba), 
        ("classifier", XGBClassifier()),
    ]
)
xgboost.fit(X_train, y_train+1)

y_xgboost = xgboost.predict(X_test)

balanced_accuracy_score(y_test+1, y_xgboost)

## Features importance

In [ ]:
result = permutation_importance(
    clf, X_test, y_test, n_repeats=10, random_state=42, n_jobs=2
)

forest_importances = pd.Series(result.importances_mean, index=X_test.columns)

fig, ax = plt.subplots()
forest_importances.plot.bar(yerr=result.importances_std, ax=ax)
ax.set_title("Feature importances using permutation on full model")
ax.set_ylabel("Mean accuracy decrease")
fig.tight_layout()
plt.show()